### 10-Minute Quick Demo of Diaspora Event SDK and Action Provider Features

#### SDK Features to Demo
| Section | SDK Features |
|---------|--------------|
| 1.1     | Register and List topics |
| 1.2     | List and update and topic configs |
| 1.3     | Produce and consume messages |
| 1.4     | Create and update triggers, get log events |

#### AP Features to Demo
| Section | AP Features |
|---------|-------------|
| 2.1     | Produce messages to AP with keys |
| 2.2     | Consume messages since a timestamp |

For a comprehensive list of SDK features, visit the [Diaspora SDK repository](https://github.com/globus-labs/diaspora-event-sdk/blob/main/DiasporaDemo.ipynb).

For a full list of AP features, refer to the [Diaspora Service documentation](https://haochenpan.github.io/diaspora-service/main/ap/examples/).

In [ ]:
from __future__ import annotations

# SDK requirement
%pip install 'diaspora-event-sdk[kafka-python]'

# AP requirement
# %pip install globus-sdk # included in diaspora-event-sdk


In [ ]:
import base64
import json
import os
import random
import time
import string
from pprint import pprint

from diaspora_event_sdk import Client as GlobusClient
from diaspora_event_sdk import KafkaConsumer, KafkaProducer, block_until_ready
from diaspora_event_sdk.version import __version__

In [ ]:
c = GlobusClient()
print('Diaspora Event SDK version:', __version__)
print("User's OpenID:", c.subject_openid)

### 1.1 Register Topics

In [ ]:
topic = 'topic-' + c.subject_openid[-12:]
print('Topic name:', topic)
print(c.register_topic(topic))

In [ ]:
random_topic_suffix = ''.join(random.choice(string.ascii_uppercase +
                                            string.digits) for _ in range(8))
random_topic_name = 'topic-' + random_topic_suffix
print('Topic name:', random_topic_name)
print(c.register_topic(random_topic_name))

In [ ]:
print(c.list_topics())

In [27]:
print('Topic to unregister:', random_topic_name)
print(c.unregister_topic(random_topic_name))

Topic to unregister: topic-FT26L8CJ
{
  "status": "no-op",
  "message": "Principal e2a8169b-feef-4d56-8eba-ab12747bee03 has no access."
}


In [28]:
print(c.list_topics())

{
  "status": "success",
  "topics": [
    "report",
    "topic-ab12747bee03"
  ]
}


### 1.2 List and Update Topic Configs

In [29]:
print(c.get_topic_configs(topic))

{
  "status": "success",
  "configs": {
    "compression.type": "producer",
    "leader.replication.throttled.replicas": "",
    "message.downconversion.enable": "true",
    "min.insync.replicas": "2",
    "segment.jitter.ms": "0",
    "cleanup.policy": "delete",
    "flush.ms": "9223372036854775807",
    "follower.replication.throttled.replicas": "",
    "segment.bytes": "1073741824",
    "retention.ms": "604800000",
    "flush.messages": "9223372036854775807",
    "message.format.version": "3.0-IV1",
    "max.compaction.lag.ms": "9223372036854775807",
    "file.delete.delay.ms": "60000",
    "max.message.bytes": "1048588",
    "min.compaction.lag.ms": "0",
    "message.timestamp.type": "CreateTime",
    "preallocate": "false",
    "min.cleanable.dirty.ratio": "0.5",
    "index.interval.bytes": "4096",
    "unclean.leader.election.enable": "true",
    "retention.bytes": "-1",
    "delete.retention.ms": "86400000",
    "segment.ms": "604800000",
    "message.timestamp.difference.max.ms

In [30]:
configs = {
    'delete.retention.ms': 43200000,
    'retention.ms': 43200000,
}
print(c.update_topic_configs(topic, configs))

{
  "status": "success",
  "before": {
    "retention.ms": "604800000",
    "delete.retention.ms": "86400000"
  },
  "after": {
    "retention.ms": "43200000",
    "delete.retention.ms": "43200000"
  }
}


In [31]:
print(c.update_topic_partitions(topic, 2))

{
  "status": "success"
}


### 1.3 Produce and Consume Messages

In [34]:
'''
    Synchronously produce messages to a registered topic.
    expected return: 
        multiple RecordMetadata(...)
'''

producer = KafkaProducer()
future = producer.send(
    topic, {'message': 'Synchronous message 1 from Diaspora SDK'})
print(future.get(timeout=10))
future = producer.send(
    topic, {'message': 'Synchronous message 2 from Diaspora SDK'})
print(future.get(timeout=10))

RecordMetadata(topic='topic-ab12747bee03', partition=1, topic_partition=TopicPartition(topic='topic-ab12747bee03', partition=1), offset=3, timestamp=1720496811111, log_start_offset=0, checksum=None, serialized_key_size=-1, serialized_value_size=54, serialized_header_size=-1)
RecordMetadata(topic='topic-ab12747bee03', partition=0, topic_partition=TopicPartition(topic='topic-ab12747bee03', partition=0), offset=1, timestamp=1720496812488, log_start_offset=0, checksum=None, serialized_key_size=-1, serialized_value_size=54, serialized_header_size=-1)


In [35]:
'''
    Asynchronously produce batched messages to a registered topic.
    See https://kafka-python.readthedocs.io/en/master/apidoc/KafkaProducer.html
    for more producer settings and usage.
    expected return: None
'''
producer = KafkaProducer()
producer.send(topic, {'message': 'Asynchronous message 3 from Diaspora SDK'})
producer.send(topic, {'message': 'Asynchronous message 4 from Diaspora SDK'})
producer.flush()

In [41]:
'''
    Consume produced messages from the beginning of the topic.
    The consumer exits in three seconds.
    If the topic has more than one partitions, messages may arrive out of order.
    See https://kafka-python.readthedocs.io/en/master/apidoc/KafkaConsumer.html 
    for more consumer settings and usage.
    expected return:
        multiple {'message': ...}
'''
consumer = KafkaConsumer(topic, auto_offset_reset='earliest')
for message in consumer:
    print(message)


## 1.4 Create and Update triggers, Get Execution Logs

### 3.0 Create a deployment package

In [42]:
trigger_package = f'{os.getcwd()}/my_deployment_package'
trigger_file = 'lambda_function.py'
trigger_name_in_def='lambda_handler'
os.system(f'mkdir {trigger_package}')

0

### 3.1 Save code to `trigger_package/trigger_file`

In [43]:
trigger_code = f'''import base64

def {trigger_name_in_def}(event, context):
    try:
        print('EVENT:')
        print(event)

        for partition, records in event['records'].items():
            for record in records:
                print("topic:", record['topic'],
                      "partition:", record['partition'],
                      "offset:", record['offset'],
                      "key:", record.get("key", "NOT-SET"),
                      "value:", base64.b64decode(record['value']))
    except Exception as e:
        print("ERROR:", e)
'''

with open(os.path.join(trigger_package, trigger_file), 'w') as f:
  f.write(trigger_code)

### 3.2 Zip the code at `trigger_file`

In [44]:
def get_zipped_code(lambda_function_package):
    print(f"Zipping {lambda_function_package}")
    os.system(f"cd {lambda_function_package} && zip -r {lambda_function_package}.zip .")
    with open(f"{lambda_function_package}.zip", "rb") as f:
        return base64.b64encode(f.read()).decode('utf-8')
zipped_code = get_zipped_code(trigger_package)

Zipping /Users/haochen/Downloads/diaspora-service/examples/my_deployment_package
  adding: lambda_function.py (deflated 55%)
